https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/03.%20Feature%20Engineering/03.%20Feature%20Engineering.ipynb

https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/04.%20Model%20Training/06.%20MT%20-%20Random%20Forest.ipynb

https://towardsdatascience.com/text-classification-in-python-dd95d264c802

https://github.com/miguelfzafra/Latest-News-Classifier/tree/master/0.%20Latest%20News%20Classifier/04.%20Model%20Training



1. Read in the data
2. Test train split
3. use TFIFD to add in the new features
4. Use an algorithm to make some predictions
5. Assess the accuracy

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_reviews = pd.read_excel("data/out/Womens Clothing E-Commerce Reviews with Emotions.xlsx")
df_reviews.dropna(inplace=True) 
df_reviews.reset_index(drop=True, inplace=True)
df_reviews.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Transformed Review Text,Transformed Review Text_wordcloud,Main Emotion,Main Emotion Score
0,850,64,A really cute top,"I am normally an xs or s in tanks, so ordered ...",4,1,31,General Petite,Tops,Blouses,"I am normally an xs or s in tanks, so ordered ...",normally xs tanks ordered size looks pretty mu...,Joy,0.588694
1,829,30,Great shirt,"I ordered this shirt last week, and i wore it ...",5,1,0,General,Tops,Blouses,"I ordered this shirt last week, and i wore it ...",ordered shirt last week wore soon arrived beau...,Joy,0.806345
2,1008,33,Great skirt,"Love the material -- it is a thick cotton, but...",5,1,4,General,Bottoms,Skirts,"Love the material -- it is a thick cotton, but...",love material thick cotton structured enough t...,Joy,0.626453
3,872,37,Lovely top,"This is a beautiful, simple, yet feminine top....",5,1,0,General,Tops,Knits,"This is a beautiful, simple, yet feminine top....",beautiful simple yet feminine top little tight...,Joy,0.693493
4,1056,26,Soft but fit oddly,"I wanted to love these, but the zipper is extr...",3,1,0,General Petite,Bottoms,Pants,"I wanted to love these, but the zipper is extr...",wanted love zipper extremely short flattering ...,Joy,0.600855


In [3]:
df_reviews['Main Emotion'].value_counts()

Joy        390
Sadness     33
Fear         9
Anger        4
Name: Main Emotion, dtype: int64

In [4]:
emotion_map = {
    'Joy': 0,
    'Sadness': 1,
    'Fear': 2,
    'Anger': 3
}

df_reviews['Main Emotion Code'] = df_reviews['Main Emotion']
df_reviews = df_reviews.replace({'Main Emotion Code': emotion_map})
df_reviews.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Transformed Review Text,Transformed Review Text_wordcloud,Main Emotion,Main Emotion Score,Main Emotion Code
0,850,64,A really cute top,"I am normally an xs or s in tanks, so ordered ...",4,1,31,General Petite,Tops,Blouses,"I am normally an xs or s in tanks, so ordered ...",normally xs tanks ordered size looks pretty mu...,Joy,0.588694,0
1,829,30,Great shirt,"I ordered this shirt last week, and i wore it ...",5,1,0,General,Tops,Blouses,"I ordered this shirt last week, and i wore it ...",ordered shirt last week wore soon arrived beau...,Joy,0.806345,0
2,1008,33,Great skirt,"Love the material -- it is a thick cotton, but...",5,1,4,General,Bottoms,Skirts,"Love the material -- it is a thick cotton, but...",love material thick cotton structured enough t...,Joy,0.626453,0
3,872,37,Lovely top,"This is a beautiful, simple, yet feminine top....",5,1,0,General,Tops,Knits,"This is a beautiful, simple, yet feminine top....",beautiful simple yet feminine top little tight...,Joy,0.693493,0
4,1056,26,Soft but fit oddly,"I wanted to love these, but the zipper is extr...",3,1,0,General Petite,Bottoms,Pants,"I wanted to love these, but the zipper is extr...",wanted love zipper extremely short flattering ...,Joy,0.600855,0


In [5]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Clothing ID                        436 non-null    int64  
 1   Age                                436 non-null    int64  
 2   Title                              436 non-null    object 
 3   Review Text                        436 non-null    object 
 4   Rating                             436 non-null    int64  
 5   Recommended IND                    436 non-null    int64  
 6   Positive Feedback Count            436 non-null    int64  
 7   Division Name                      436 non-null    object 
 8   Department Name                    436 non-null    object 
 9   Class Name                         436 non-null    object 
 10  Transformed Review Text            436 non-null    object 
 11  Transformed Review Text_wordcloud  436 non-null    object 

In [7]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', ngram_range=ngram_range, stop_words=None, lowercase=False, max_df=max_df, min_df=min_df, max_features=max_features, norm='l2', sublinear_tf=True)

X = df_reviews['Transformed Review Text_wordcloud']
y = df_reviews['Main Emotion Code']

tfid_features = tfidf_vectorizer.fit_transform(X).toarray()


In [59]:
def get_correlated_ngrams(emotion_code):

    features_chi2 = chi2(tfid_features, y == emotion_code) # For the current emotion, use chi2 stats to evaluate the highest correlations with the target
    indices = np.argsort(features_chi2[0])[::-1] # np.argsort returns the indices that would sort the array ascending, [::-1] makes it descending with the highest correlation in first place
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices] # get_feature_names uses the sorted indices to order the features most to least correlated with the target
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1] # Selects the feature names that are single words
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2] # Selects the feature names that are two words
    
    return unigrams, bigrams

In [61]:
for emotion, emotion_code in sorted(emotion_map.items()):
    unigrams, bigrams = get_correlated_ngrams(emotion_code)
    
    print(f"{emotion}:")
    print(f"Most correleated unigrams: {', '.join(unigrams[:20])}")
    print(f"Most correleated bigrams: {', '.join(bigrams[:2])}\n")

Anger:
Most correleated unigrams: body, felt, wash, small, looking, retailer, unfortunately, could, detail, put, usual, wardrobe, waist, away, way, tank, maybe, around, made, shoulders
Most correleated bigrams: fit perfectly, true size

Fear:
Most correleated unigrams: area, yet, normal, full, stunning, pull, took, retailer, works, xxs, overall, absolutely, medium, came, either, piece, first, already, might, body
Most correleated bigrams: well made, feel like

Joy:
Most correleated unigrams: return, retailer, tight, order, area, sizes, two, pull, put, bust, however, runs, body, another, away, perfect, maybe, might, full, also
Most correleated bigrams: love dress, cannot wait

Sadness:
Most correleated unigrams: return, order, tight, another, two, decided, also, pink, would, put, sizes, bust, front, away, however, maybe, perfect, around, cut, got
Most correleated bigrams: true size, love dress



In [57]:
from sklearn.feature_selection import chi2
import numpy as np

for emotion, emotion_code in sorted(emotion_map.items()):
    features_chi2 = chi2(tfid_features, y == emotion_code) # For the current emotion, use chi2 stats to evaluate the highest correlations with the target
    indices = np.argsort(features_chi2[0])[::-1] # np.argsort returns the indices that would sort the array ascending, [::-1] makes it descending with the highest correlation in first place
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices] # get_feature_names uses the sorted indices to order the features most to least correlated with the target
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1] # Selects the feature names that are single words
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2] # Selects the feature names that are two words
    
    print(f"{emotion}:")
    print(f"Most correleated unigrams: {', '.join(unigrams[:20])}")
    print(f"Most correleated bigrams: {', '.join(bigrams[:2])}\n")
    

Anger:
Most correleated unigrams: body, felt, wash, small, looking, retailer, unfortunately, could, detail, put, usual, wardrobe, waist, away, way, tank, maybe, around, made, shoulders
Most correleated bigrams: fit perfectly, true size

Fear:
Most correleated unigrams: area, yet, normal, full, stunning, pull, took, retailer, works, xxs, overall, absolutely, medium, came, either, piece, first, already, might, body
Most correleated bigrams: well made, feel like

Joy:
Most correleated unigrams: return, retailer, tight, order, area, sizes, two, pull, put, bust, however, runs, body, another, away, perfect, maybe, might, full, also
Most correleated bigrams: love dress, cannot wait

Sadness:
Most correleated unigrams: return, order, tight, another, two, decided, also, pink, would, put, sizes, bust, front, away, however, maybe, perfect, around, cut, got
Most correleated bigrams: true size, love dress



In [42]:
from sklearn.feature_selection import chi2
import numpy as np

for emotion, emotion_code in sorted(emotion_map.items()):
    features_chi2 = chi2(tfid_features, y == emotion_code) # For the current emotion, use chi2 stats to evaluate the highest correlations with the target
    indices = np.argsort(features_chi2[0]) # np.argsort returns the indices that would sort the array !!!IN LOWEST TO HIGHEST ORDER!!!
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices] # get_feature_names uses the sorted indices to order the features most to least correlated with the target
    
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1] # Selects the feature names that are single words
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2] # Selects the feature names that are two words

    #print(f"{emotion}:")
    #print(f"Most correleated unigrame: {', '.join(unigrams[-5:])}")
    print("# '{}' category:".format(emotion))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Anger' category:
  . Most correlated unigrams:
. looking
. small
. wash
. felt
. body
  . Most correlated bigrams:
. true size
. fit perfectly

# 'Fear' category:
  . Most correlated unigrams:
. stunning
. full
. normal
. yet
. area
  . Most correlated bigrams:
. feel like
. well made

# 'Joy' category:
  . Most correlated unigrams:
. area
. order
. tight
. retailer
. return
  . Most correlated bigrams:
. cannot wait
. love dress

# 'Sadness' category:
  . Most correlated unigrams:
. two
. another
. tight
. order
. return
  . Most correlated bigrams:
. love dress
. true size



In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_reviews['Transformed Review Text_wordcloud'], 
                                                    df_reviews['Main Emotion Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [10]:
X_train = df_reviews['Transformed Review Text_wordcloud']
y_train = df_reviews['Main Emotion Code']

In [22]:
pd.DataFrame(tfid_features)

,0,1,2,3,4,5,6,7,8,9,...,263,264,265,266,267,268,269,270,271,272
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.202146,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.180455,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.247227,0.000000,0.0,0.000000,0.0,0.170473,0.000000,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.189198,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.141528,0.000000,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.366034
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.305821,0.0,0.000000,0.0,0.236998,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.176293,0.000000,0.0,0.0,0.000000
432,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.237496,0.254225,0.0,...,0.000000,0.000000,0.0,0.210165,0.0,0.000000,0.212011,0.0,0.0,0.267200
433,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.149616,0.000000,0.0,0.0,0.000000
434,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.197121,0.000000,0.0,0.000000,0.0,0.135923,0.000000,0.0,0.0,0.000000


In [ ]:
#features_train = tfidf.fit_transform(X_train).toarray()
#labels_train = y_train
#print(features_train.shape)

#features_test = tfidf.transform(X_test).toarray()
#labels_test = y_test
#print(features_test.shape)

In [21]:
features_chi2[2]

IndexError: tuple index out of range

sklearn.feature_extraction.text.TfidfVectorizer

In [16]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(emotion_map.items()):
    features_chi2 = chi2(tfid_features, y == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Anger' category:
  . Most correlated unigrams:
. looking
. small
. wash
. felt
. body
  . Most correlated bigrams:
. true size
. fit perfectly

# 'Fear' category:
  . Most correlated unigrams:
. stunning
. full
. normal
. yet
. area
  . Most correlated bigrams:
. feel like
. well made

# 'Joy' category:
  . Most correlated unigrams:
. area
. order
. tight
. retailer
. return
  . Most correlated bigrams:
. cannot wait
. love dress

# 'Sadness' category:
  . Most correlated unigrams:
. two
. another
. tight
. order
. return
  . Most correlated bigrams:
. love dress
. true size



In [88]:
type(tfifd_features)

numpy.ndarray

In [89]:
X = pd.concat([df_reviews.iloc[:,[1,4,5,6]], pd.DataFrame(tfifd_features)], axis=1)
y = df_reviews["Main Emotion Code"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
type(features_train)

numpy.ndarray

In [82]:
features_train.shape

(370, 244)

In [84]:
labels_train.shape

(370,)

In [76]:
tfifd_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [94]:
tfifd_features.shape

(436, 273)

In [95]:
y.shape

(436,)

In [90]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(emotion_map.items()):
    print(Product, category_id)
    features_chi2 = chi2(tfifd_features, labels_train == category_id)
#    indices = np.argsort(features_chi2[0])
#    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
#    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#    print("# '{}' category:".format(Product))
#    print("\nMost correlated unigrams:\n{}".format('\n'.join(unigrams[-15:])))
#    print("\nMost correlated bigrams:\n{}".format('\n'.join(bigrams[-5:])))
#    print("")

Anger 3


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 436 is different from 370)

In [69]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(emotion_map.items()):
    features_chi2 = chi2(tfifd_features, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#    trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
    print("# '{}' category:".format(Product))
    print("\nMost correlated unigrams:\n{}".format('\n'.join(unigrams[-15:])))
    print("\nMost correlated bigrams:\n{}".format('\n'.join(bigrams[-5:])))
#    print("\nMost correlated trigrams:\n{}".format('\n'.join(trigrams[-2:])))
    print("")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 436 is different from 348)

get rid of full stops